In [127]:
import pandas as pd

In [128]:
api_root = 'https://one.nhtsa.gov/webapi/api/SafetyRatings'
api_format = '?format=csv'
api_step = ''

def build_api_call(api_root, api_format, api_step = None):
    api_call = ''.join([api_root, api_step, api_format])
    return api_call.replace(" ", "%20")

In [129]:
api_call = build_api_call(api_root, api_format, api_step)
model_yrs = pd.read_csv(api_call)

In [130]:
min_yr = 2017

In [131]:
process_model_yrs = model_yrs[model_yrs.MODEL_YR >= min_yr]

In [132]:
api_step = '/modelyear/'

In [133]:
def build_makes(x, api_step):
    api_step = ''.join([api_step, str(x)])
    api_call = build_api_call(api_root, api_format, api_step)
    return pd.read_csv(api_call)

makes = pd.concat(list(process_model_yrs.MODEL_YR.apply(build_makes, args=(api_step,))))

In [134]:
def build_model_apis(x):
    api_step = ''.join(['/modelyear/', str(x.MODEL_YR), '/make/', x.MAKE])
    api_call = build_api_call(api_root, api_format, api_step)
    x['api'] = api_call
    return x

process_makes = makes.apply(build_model_apis, 1)

In [135]:
def get_model_info(x):
    vars_to_keep = ['MODEL_YR', 'MAKE', 'MODEL']
    
    try:
        df = pd.read_csv(x.api)
        x['status'] = 'success'
        # print ' '.join(['SUCCESS:', str(x.MODEL_YR), str(x.MAKE)])
        df = df[vars_to_keep]
        x['data'] = df
    except:
        x['status'] = 'fail'
        print ' '.join(['FAIL:', str(x.MODEL_YR), str(x.MAKE)])

    return x

In [136]:
raw_models = process_makes.apply(get_model_info, 1)

In [137]:
models = pd.concat(list(raw_models.data))
raw_models[raw_models.status == 'fail']

,MODEL_YR,MAKE,MODEL,MMY,VEHICLE_ID,api,status,data


In [138]:
def build_trim_apis(x):
    api_step = ''.join(['/modelyear/', str(x.MODEL_YR), '/make/', x.MAKE, '/model/', str(x.MODEL)])
    api_call = build_api_call(api_root, api_format, api_step)
    x['api'] = api_call
    return x

process_models = models.apply(build_trim_apis, 1)

In [139]:
def get_trim_info(x):
    vars_to_keep = ['MODEL_YR', 'MAKE', 'MODEL', 'MMY', 'VEHICLE_ID']
    
    try:
        df = pd.read_csv(x.api)
        x['status'] = 'success'
        # print ' '.join(['SUCCESS:', str(x.MODEL_YR), str(x.MAKE)])
        df = df[vars_to_keep]
        df['MAKE'] = x.MAKE
        df['MODEL_YR'] = x.MODEL_YR
        df['MODEL'] = x.MODEL
        x['data'] = df
    except:
        x['status'] = 'fail'
        print ' '.join(['FAIL:', str(x.MODEL_YR), str(x.MAKE), str(x.MODEL)])

    return x

In [140]:
raw_trims = process_models.apply(get_trim_info, 1)

FAIL: 2018 ACURA RLX SPORT HYBRID 
FAIL: 2018 MERCEDES-BENZ E-CLASS 
FAIL: 2018 MERCEDES-BENZ S-CLASS 
FAIL: 2018 PORSCHE MACAN 
FAIL: 2017 ACURA MDX SPORT HYBRID 
FAIL: 2017 ACURA RLX SPORT HYBRID 
FAIL: 2017 MERCEDES-BENZ CL-CLASS  
FAIL: 2017 MERCEDES-BENZ E-CLASS 
FAIL: 2017 MERCEDES-BENZ S-CLASS 
FAIL: 2017 PORSCHE MACAN 


In [141]:
trims = pd.concat(list(raw_trims[raw_trims.status == 'success'].data))
failed_models = raw_trims[raw_trims.status == 'fail']


In [142]:
def build_vehicle_apis(x):
    api_step = ''.join(['/VehicleId/', str(x.VEHICLE_ID)])
    api_call = build_api_call(api_root, api_format, api_step)
    x['api'] = api_call
    return x

process_trims = trims.apply(build_vehicle_apis, 1)

In [143]:
def get_vehicle_info(x):
    
    try:
        df = pd.read_csv(x.api)
        x['status'] = 'success'
        # print ' '.join(['SUCCESS:', str(x.MODEL_YR), str(x.MAKE)])
        x['data'] = df
    except:
        x['status'] = 'fail'
        print ' '.join(['FAIL:', str(x.MODEL_YR), str(x.MAKE), str(x.MODEL)])

    return x

In [144]:
raw_vehicles = process_trims.apply(get_vehicle_info, 1)

In [145]:
vehicles = pd.concat(list(raw_vehicles[raw_vehicles.status == 'success'].data))

In [146]:
vehicles.to_csv('vehicle_safety_info.csv')